# SDK Examples - KPIs

Examples for new KPI functions that were released with smsdk v1.1

*created April 2023*

In [ ]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd
import os

In [ ]:
env_var_tenant = 'ENV_SDK_VAR_TENANT'
env_var_api_key = 'ENV_SDK_VAR_API_KEY'
env_var_api_secret = 'ENV_SDK_VAR_API_SECRET'

# Check if the environment variable exists
if env_var_tenant in os.environ:
    # Retrieve the value of the environment variable
    tenant = os.environ[env_var_tenant]
else:
    # Use a default value if the environment variable is not present
    tenant = 'demo'

# Check if the environment variable exists
if env_var_api_key in os.environ:
    # Retrieve the value of the environment variable
    api_key = os.environ[env_var_api_key]
else:
    # Use a default value if the environment variable is not present
    api_key = ''

# Check if the environment variable exists
if env_var_api_secret in os.environ:
    # Retrieve the value of the environment variable
    api_secret = os.environ[env_var_api_secret]
else:
    # Use a default value if the environment variable is not present
    api_secret = ''

In [ ]:
cli = client.Client(tenant)
cli.login('apikey', 
          key_id = api_key, 
          secret_id = api_secret)

machine_types = cli.get_machine_type_names()
machine_types

# KPIs
---

The SDK has three functions related to KPIs. The first returns a list of all availible KPIs. The second of which allows a user to see which KPIs are availible for a particular asset. The third makes use of our Data Visualization API which allows a user to see these KPIs over a timeframe.

## Get KPIs

```cli.get_kpis()```

List the available KPIs for all assets.

In [ ]:
kpis_dict = cli.get_kpis()
kpis_dict

In [ ]:
# example: create a table with KPIs and formulas
df_kpis = pd.DataFrame(kpis_dict)
df_kpis

## Get KPIs for Asset

```cli.get_kpis_for_asset(**asset_selection)```

List the available KPIs for a specific asset. For asset selection, see the example below or [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/asset_selection.md) GitHub doc.

**NOTE: machine type names must be the raw ones, not the clean/display names printed above!!!**

In [ ]:
# get the raw machine type names (requires a sort of roundabout method)
def get_raw_machine_type(clean_machine_type: str) -> str:
    machines = cli.get_machine_names(source_type=clean_machine_type)
    return(cli.get_type_from_machine(machines[0]))

In [ ]:
# KPIs by machine type
machine_types_clean = ['Pick & Place', 'Diecast']
asset1 = { 
    "asset_selection": {
        'machine_type': [get_raw_machine_type(i) for i in machine_types_clean]
    }
}
kpis_asset1 = cli.get_kpis_for_asset(**asset1)
pd.DataFrame(kpis_asset1)

In [ ]:
# check machines of some type
prod_status_machines = cli.get_machine_names(source_type='Pick & Place')

# KPIs by machine
asset2 = {
    'asset_selection': {
        'machine_type': [get_raw_machine_type("Pick & Place")],
        'machine_source': [prod_status_machines[0]]
    }
}
kpis_asset2 = cli.get_kpis_for_asset(**asset2)
pd.DataFrame(kpis_asset2)

## Get KPIs over Time

```cli.get_kpi_data_viz(machine_source, kpis, i_vars, time_selection, **optional_data_viz_query)```

Get the value of a chosen KPI over some specific period of time. See examples of how to use all the parameters below. Find additional info about parameter formatting on [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/data_viz_query.md) GitHub doc.

```machine_source```: a list of machine names

In [ ]:
this_machine_sources = [prod_status_machines[0]]

```kpis```: a list of names of kpis that you want data for

In [ ]:
# query all KPIs - it should return only those available for this machine
this_kpis = df_kpis["name"].to_list()

```i_vars```: a list of dicts, one per independent (x-axis) variable -- for our purposes, usually a list of size one, where the one dict has details about some time field

In [ ]:
this_i_vars = [
    {
      "name": "endtime",  # name of independent variable
      "time_resolution": "day",   # choose anything from 'year' down to 'second'
      "query_tz": "America/Los_Angeles",    # optional - timezone that query is in
      "output_tz": "America/Los_Angeles",  # optional - timezone that returned data should be in
      "bin_strategy": "user_defined2",  # optional - how to bin the data ('user_defined2', 'none', or 'categorical')
      "bin_count": 50   # optional - how many bins you want the data in
    }
]

```time_selection```: a dictionary for time selection, with all the same options as MA Data Viz (see [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/data_viz_query.md#time_selection) GitHub doc for more specifics)

In [ ]:
# relative time selection
this_time_selection = {
    "time_type": "relative",
    "relative_start": 7,
    "relative_unit": "year",
    "ctime_tz": "America/Los_Angeles"
}

# absolution time selection
# this_time_selection = {
#   "time_type": "absolute",
#   "start_time": "2023-02-23T08:00:00.000Z",
#   "end_time": "2023-03-01T21:35:35.499Z",
#   "time_zone": "America/Los_Angeles"
# }

This function may take a minute to run. If you run into issues, make sure the KPI you're looking for exists for the asset you're looking for using get_kpis_for_asset (above). Also, try creating the same query in MA. If it works, use the Developer Panel of your browser to view the contents of the API call.

In [ ]:
cli.get_kpi_data_viz(machine_sources=this_machine_sources,
                     kpis=this_kpis, i_vars=this_i_vars,
                     time_selection=this_time_selection)

Using additional args - Where

In [ ]:
# this_args = {'where': [{
#     "name": "oee",
#     "op": "eq",
#     "value": "65.7489705585684"
# }]
# }

this_args = {
    "where": [],
}

data_dict = cli.get_kpi_data_viz(machine_sources=this_machine_sources,
                     kpis=this_kpis, i_vars=this_i_vars,
                     time_selection=this_time_selection,
                     **this_args)

if len(data_dict):
    print(data_dict[0])

In [ ]:
# example: extract a table
data_list = []
for point in data_dict:
    new_row = {}
    # extract i_vals (assuming it's just one - time)
    new_row["endtime"] = point["i_vals"]["endtime"]["bin_avg"]
    # extract d_vals
    for d in point["d_vals"].keys():
        new_row[d] = point["d_vals"][d]["avg"]
    # extract other (dependent) values
    for kpi in point["kpi_dependencies"].keys():
        for field in point["kpi_dependencies"][kpi].keys():
            new_row[field] = point["kpi_dependencies"][kpi][field]
    # add to list of rows
    data_list.append(new_row)
df_kpis = pd.DataFrame(data_list)
        
# now you can graph the data using this table

df_kpis.head()